In [ ]:
import pandas as pd
import numpy as np

def log_transf(x):
  if x != 0:
    return np.log(x)
  if x == 0:
    return 0

%load_ext rpy2.ipython

In [ ]:
state_list = list('WA','AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA',
                  'HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI',
                  'MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND',
                  'OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA',
                  'WV','WI','WY')

In [ ]:
df_US = pd.read_csv('/USCOVID_BY_STATE.csv')
i = 1
for states in state_list:
  print(i)
  df_current_state = df_US[df_US['state'] == states]
  df_current_state.index = df_current_state['date']
  df_current_state = df_current_state[all_list]
  df_current_state = df_current_state.replace(np.nan, 0)
  df_current_state[df_current_state < 0] = 0
  df_current_state.to_csv('/'+states+'_whole.csv')
  i = i + 1

In [ ]:
def new_data_frame(col_core, h, states):
  df_US = pd.read_csv('/'+states+'/whole.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  df_US = df_US.drop(columns=['date'])
  df_US_log = pd.DataFrame()
  for col in df_US.columns:
    df_US_log[col] = df_US[col].apply(lambda x: log_transf(x))
  df_state_cases = df_US_log.copy()

  df_lag = h

  df_positive1 = pd.DataFrame(df_state_cases[col_core][df_lag:])
  n = df_positive1.shape[0] + df_lag
  # lag 1
  df_positive1['date'] = df_positive1.index
  df_positive1.index = np.arange(1,n-(df_lag-1))
  for col in df_state_cases.columns:
    newcol = df_state_cases[col][:-df_lag]
    newcol.index = np.arange(1,n-(df_lag-1))
    df_positive1[col + '_l' + str(df_lag)] = newcol
  df_positive1.index = df_positive1['date']
  df_positive1 = df_positive1.drop(columns=['date'])

  df_positive_new_all1 = df_positive1.copy()

  for i in range(13):
    llag = i + 2
    df_positive2 = pd.DataFrame(df_state_cases[col_core][(df_lag+(llag-1)):])
    df_positive2['date'] = df_positive2.index
    df_positive2.index = np.arange(1,(n-(df_lag+(llag-2))))
    for col in df_state_cases.columns:
      newcol = df_state_cases[col][:-(df_lag+(llag-1))]
      newcol.index = np.arange(1,(n-(df_lag+(llag-2))))
      df_positive2[col + '_l' + str(df_lag+(llag-1))] = newcol

    df_positive2.index = df_positive2['date']
    df_positive2 = df_positive2.drop(columns=['date'])
    df_positive2 = df_positive2.drop(columns=[col_core])

    df_positive_new_all1 = pd.concat([df_positive_new_all1, df_positive2], axis=1)
    df_positive_new_all1 = df_positive_new_all1.replace(np.nan, 0)
  return df_positive_new_all1

In [ ]:
def create_and_save_files(col_core, h, states):
  df_US = pd.read_csv('/'+states+'/whole.csv')
  df_US.index = pd.to_datetime(df_US['date'])
  df_US = df_US.drop(columns=['date'])
  df_index = pd.read_csv('/US_COVID_BY_STATE_deathIncrease.csv')
  index_total = pd.to_datetime(df_index['Unnamed: 0'])
  df_US_total = df_US.loc[index_total]
  df_new = new_data_frame(col_core, h, states)
  df_data_log_total = df_new.loc[index_total]
  df_US_train = df_US_total[0:-112]
  df_US_test = df_US_total[-112:]
  n_train = len(df_US_train)
  n_test = len(df_US_test)
  n_iter = n_test-h+1
  for i in range(int(n_iter)):
    dataset = df_data_log_total[int(i):int(i + n_train + h)]
    current_train = dataset[0:int(n_train)]

    corrs = []
    for col in current_train.columns:
      corr, _ = pearsonr(current_train[col_core], current_train[col])
      corrs.append(corr)
    df_positive_new_corr = pd.DataFrame(corrs).transpose()
    df_positive_new_corr.columns = current_train.columns
    df = df_positive_new_corr.copy()

    top_n = 8
    features = [df.T[col].nlargest(top_n).index.tolist() for n, col in enumerate(df.T)][0]
    new_set = dataset[features]
    new_set.to_csv('/'+ states +'/' + str(h)+'/'+ col_core + '_' + str(i+1) + '_log.csv')

In [ ]:
%%R
library(forecast)
library(MARSS)
library(MASS)
library(glmnet)
library(ncvreg)

invert_transformation <- function(df_train, df_forecast, period) {
  n = length(df_forecast)
  for (i in 1:n) {
    nn = length(df_train)
    current_value = df_forecast[i] + df_train[nn-(period-1)]
    df_train = append(df_train, current_value)
  }
  m = length(df_train)
  df_forecast = df_train[(m-n+1):m]
  return(df_forecast)
}

output_current_window <- function(col_core, h, window, states){
  mode <- 'diff(log,7)'
  order <- 1
  m <- 7
  
  n_train <- 185
  n_test <- 112
  n_iter <- n_test-h+1
  
  date_list <- c()
  pred_mcp <- c()
  
  df_reg_all <- read.csv(paste('~/',states,'/',h,'/',col_core,'_',window,'_log.csv', sep=''))
  n = nrow(df_reg_all)
  df_train_all <- df_reg_all[1:n_train,]
  df_test_all <- df_reg_all[(n_train+1):n,]
  n_test_current <- nrow(df_test_all)
  
  #### Univariate analysis ####
  df_train_uni <- df_train_all[,2]
  test_date <- df_test_all$date
  true_value <- df_test_all[,2]
  
  #### Multivariate models ####
  df_reg <- df_reg_all
  
  a = 7
  b = a + 1
  c = a + 2
  
  df_reg_train <- df_train_all
  df_reg_test <- df_test_all
  
  df_reg_diff <- as.data.frame(lapply(df_reg[,2:c], diff, lag=m, difference=order))
  df_reg_diff['date'] <- df_reg[-c(seq(1,m)),]$date
  
  df_reg_train_diff <- df_reg_diff[1:(nrow(df_reg_diff)-n_test_current),]
  df_reg_test_diff <- df_reg_diff[(nrow(df_reg_diff)-(n_test_current-1)):nrow(df_reg_diff),]
  
  ## Regularization
  X_train <- model.matrix(as.formula(paste(col_core, "~.", collapse = "+")),df_reg_train_diff[,1:b])[,-1]
  y_train <- df_reg_train_diff[,1]
  X_test <- as.matrix(df_reg_test_diff[,2:b])
  y_test <- df_reg_test_diff[,1]
  
  # MCP
  cv_output3 <- cv.ncvreg(X_train, y_train, gamma = 2, penalty = "MCP", nfolds = 10)
  best_lam3 <- cv_output3$lambda.min
  pred_mcp1 <- predict(cv_output3, X = X_test)

  date_list <- test_date
  pred_mcp <- invert_transformation(df_train_uni, pred_mcp1, m)

  df_forecast_R <- cbind.data.frame(date_list, exp(pred_mcp))
  
  return(df_forecast_R)
}

output_final_results = function(col_core, h, states) {
  n_train <- 185
  n_test <- 112
  n_iter <- n_test-h+1
  
  n_row = 112
  n_col = 2
  
  df_results = data.frame(matrix(ncol = n_col, nrow = n_row))
  colnames(df_results) <- c('date','MCP')
  
  if (h==1) {
    for (i in 1:n_iter) {
      df_results[i,] = output_current_window(col_core, h, i, states) 
    }
  }
  
  if (h==2) {
    i = 1
    df_results[i,] = output_current_window(col_core, h, i, states)[1,]
    df_results[i+(h-1),] = output_current_window(col_core, h, i, states)[h,]
    
    for (i in 1:n_iter) {
      df_results[i+(h-1),] = output_current_window(col_core, h, i, states)[h,]
    }
  }
  
  if (h >= 3) {
    for (i in 1:(h-1)){
      df_results[i,] = output_current_window(col_core, h, i, states)[1,]
    }
    
    for (i in 1:n_iter) {
      df_results[i+(h-1),] = output_current_window(col_core, h, i, states)[h,]
    }
  }
  return(df_results)
}

combine_results = function(col_core, states){
  df_results = data.frame(matrix(ncol = 15, nrow = 112))
  colnames(df_results) <- c('date','h=1','h=2','h=3','h=4','h=5','h=6',
                            'h=7','h=8','h=9','h=10','h=11','h=12','h=13',
                            'h=14')
  h = 1
  outputdata = output_final_results(col_core, h, states)
  df_results[,1] = outputdata$date
  df_results[,2] = outputdata$MCP
  
  for (h in 2:14) {
    outputdata = output_final_results(col_core, h, states)
    df_results[,h+1] = outputdata$MCP
  }
  
  return(df_results)
}

col_core = 'deathIncrease'
for (states in state_list) {
  print(states)
  df_results = combine_results(col_core, states)
  write.csv(df_results, paste('~/', states,'_R.csv', sep=''))
}